<a href="https://colab.research.google.com/github/GuShanzhi/Chinese-alpaca-lora/blob/main/notebook/evaluation_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "silk-road/luotuo-lora-7b-1.0")

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
import functools
import time

def log_time(func):
    @functools.wraps(func)
    def timer(*args, **kwargs):
        func_name = func.__qualname__
        start_time = time.time()
        result = func(*args, **kwargs)
        cost_time = time.time() - start_time
        cost_minute = cost_time // 60
        cost_second = cost_time - (cost_minute * 60)
        print(f"{func_name} costs time: {int(cost_minute)}m {cost_second:.2f}s\n")
        return result
    return timer

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

@log_time
def evaluate(instruction, input=None):
    print(f"Instruction: {instruction}")
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [ ]:
sentence_list = [
    "网元是什么意思",
    "帮我生成一条关于春游的团建宣传标语",
    "我要你帮我创建一个割接核查任务：我会给你输入一批网元，你要帮我检查输入的网元是否有告警，链路是否正常，端口的光功率是否异常，网元上的业务是否有告警。请你列出需要检查的诊断项以及任务步骤"
]

for sentence in sentence_list:
    evaluate(sentence)